# 단어 표현 

- One Hot Encoding : 텍스트 마이닝 이전, 단어를 Matrix 형태로 표현 
- CBOW : 텍스트 마이닝 이전, 단어를 Matrix 형태로 표현
- Skip-Gram : 주변 단어에 대한 예측 및 유사도 계산

## TF-IDF (Term Frequence Inverse Document Frequency)

- CBOW 방식을 보완하기 위해 개별 문서나, 문장에 자주 나타는 단어에 가중치를 두되, 전체적으로 많이 등장하는 단어에는 패널티를 부여해 확률 값으로 단어를 표현
- 특정 문장에서 많이 등장하는 단어는 중요한 단어가 되지만, 다른 문장에서도 많이 사용한다면, 범용적으로 사용되는 단어일 가능성이 높음
- 각 문장의 특징을 더 명확하게 구분 짓는 기법 -> 텍스트 마이닝 (분류) 높은 성능을 발휘 

In [2]:
text1 = """ 아무리 우겨봐도 어쩔 수 없네
저기 개똥 무덤이 내 집인걸
가슴을 내밀어도 친구가 없네
노래하던 새들도 멀리 날아가네
가지 마라 가지 마라 가지 말아라
나를 위해 한 번만 노래를 해주렴
나나 나나나나 쓰라린 가슴 안고
오늘 밤도 그렇게 울다 잠이 든다

마음을 다 주어도 친구가 없네
사랑하고 싶지만 마음뿐인걸
나는 개똥벌레 어쩔 수 없네
손을 잡고 싶지만 모두 떠나가네
가지 마라 가지 마라 가지 말아라
나를 위해 한 번만 손을 잡아주렴
아아 외로운 밤 쓰라린 가슴 안고
오늘 밤도 그렇게 울다 잠이 든다
울다 잠이 든다
울다 잠이 든다"""

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Kkma

In [5]:
kkma = Kkma()
sent1 = kkma.sentences(text1) # 하나의 문단을 문장으로 분해 

In [6]:
tfidf_model = TfidfVectorizer()
x = tfidf_model.fit_transform(sent1) # TF-IDF 기법으로 해석 

In [8]:
word_feature = tfidf_model.get_feature_names_out() # 각 문장의 단어를 분해 

In [10]:
word_tfidf = x.toarray() # 각 문장별 단어의 확률 값을 확인 

In [11]:
import pandas as pd 

In [ ]:
pd.DataFrame(data=word_tfidf, columns=word_feature)

# 문장 표현 (Sentence Representation)
- 텍스트 분류 및 연관분석 등, 텍스트 마이닝 기법을 사용하기 전, 문장단위의 표현 방법
- Text to Sequence : 여러 문장을 컴퓨터가 이해 할 수 있는 Sequence로 표현
- Padding : 변환된 Sequence를 똑같은 형태로( Column의 수를 지정)하여 변환하는 작업  

In [15]:
# 기업의 한줄평을 X / 기업추천여부 Y  -> 분류 모델 생성 (Text Mining + 문장 표현) 
df1 = pd.read_csv('24_Data.csv')
df2 = df1.dropna()

**Text to Sequence**

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [19]:
token_model = Tokenizer()
token_model.fit_on_texts(df2['기업한줄평']) # 기업 한줄 평 내 모든 단어 토큰을 하나의 정수로 매칭

In [22]:
df2['기업한줄평'][0]

'"부서 by 부서, 팀장 by 팀장으로 근무환경이 달라지는 곳"'

In [24]:
text_seq = token_model.texts_to_sequences(df2['기업한줄평']) # 텍스트를 매칭된 숫자로 변환 
text_seq[0]

[28, 29, 28, 87, 29, 88, 89, 90, 3]

In [26]:
# token_model.word_index # 어떤 문자가 어떤 숫자로 매칭 되었는가 표현 

**Padding**
- 텍스트 길이에 따라 Sequence 길이가 달라지는데, 이 서로다른 길이를 0으로 채워 나눠진 문장을 일정한 크기의 Matrix 표현 

In [28]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [33]:
X = pad_sequences(text_seq, maxlen=50, padding='post')
# pd.DataFrame(X)

**Text Mining**

In [ ]:
df2['Target'] = df2['기업추천여부'].replace({'추천':1, '비추천':0})
df2['Target'].value_counts()

In [37]:
X = df2['기업한줄평']
Y = df2['Target']

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state=1234)

In [41]:
# 파이프라인을 구성하여, Text to Seq / Padding 수행 
# 특정 데이터를 전처리하는 함수를 생성해 파이프라인 안에 넣어 처리 
def text_preprocessing(X):
    token_model = Tokenizer()
    token_model.fit_on_texts(X) # 입력 받은 X로 단어 사전을 구성 
    seq_x = token_model.texts_to_sequences(X) # 입력 받은 X 문자가 Seq 형태로 변환
    return pad_sequences(seq_x, maxlen=20, padding='post')

In [44]:
# 사용자가 직접 생성한 전처리 함수를 PipeLine 내에서 사용할 수 있게 변환 
from sklearn.preprocessing import FunctionTransformer

In [47]:
text_token_tranfer  = FunctionTransformer(text_preprocessing)

In [48]:
# 파이프 라인을 구성하여 학습 
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling  import SMOTE
from sklearn.preprocessing   import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble        import RandomForestClassifier

In [50]:
# 학습 파이프라인을 구성 
model_pipe = make_pipeline(text_token_tranfer, MinMaxScaler(), 
                           SMOTE(), RandomForestClassifier() )

In [52]:
# 교차검증 및 하이퍼파라미터 튜닝 
hyper_list = {'randomforestclassifier__max_depth':[5,10,15,20],
              'randomforestclassifier__min_samples_split':range(5,10),
              'randomforestclassifier__criterion':['gini','entropy']}
gird_model = GridSearchCV(model_pipe, param_grid=hyper_list, 
                          cv=3, scoring='f1', n_jobs=-1)
gird_model.fit(X_train, Y_train)
best_model = gird_model.best_estimator_

In [53]:
from sklearn.metrics import classification_report

In [54]:
Y_train_pred = best_model.predict(X_train)
Y_test_pred  = best_model.predict(X_test)

In [ ]:
print(classification_report(Y_train, Y_train_pred))

In [ ]:
print(classification_report(Y_test, Y_test_pred))

In [58]:
# 새로운 데이터 입력 
new_Data = "회사의 협력적 관계에 있어서 가장 중요한건 팀워크입니다만?ㅋㅋ 우리회산 그게 잘 안되는 듯 ㅋㅋ"
input_data = pd.DataFrame(data=[new_Data], columns=['기업한줄평'] )

In [ ]:
best_model.predict(input_data) # 기업 추천 여부 예측 

In [63]:
best_model.predict_proba(input_data) # 기업 추천 여부 확률을 예측 

array([[0.25236905, 0.74763095]])

# 신경망 알고리즘을 이용한 자연어 처리 

## RNN (Recurrent Neural Network / 순환신경망 알고리즘)
- 순서를 갖는 Sequence 데이터를 처리 / 현재 까지 처리된 정보 상태(State) 저장하여, 앞/뒤 정보를 유지하며 학습을 수행
    - ![image.png](https://wikidocs.net/images/page/22886/rnn_image3.5.PNG)
    - 1. 문장 표현 Sequence가 순차적으로 Node에 들어감
    - 2. 각 Node가 선행 단어부터 순차적으로 처리 -> 앞서 처리된 단어의 결과를 이용해, 다음층으로 보낼 가중치를 계산
    - 3. 각 단어의 위치에서 Layer(Node)통과한 결과가 출력

- 특징 :
  - 데이터의 순서와 서열을 반영하여 학습
  - 장기 의존성 문제 (Long Term Dependencies) : 시퀀스가 길어지면, 이전에 처리된 정보가 점점 학습이 진행되며 손실
    - Gradient 소실 / 폭주 

In [64]:
from keras.layers import SimpleRNN
from keras.layers import Embedding 
from keras.models import Sequential 
from keras.layers import Dense

In [66]:
X_train_rnn = text_preprocessing(X_train)
X_test_rnn = text_preprocessing(X_test)

In [ ]:
model_RNN = Sequential()
model_RNN.add(Embedding(10000, 8))
model_RNN.add(SimpleRNN(32))
model_RNN.add(Dense(1, activation='sigmoid'))
model_RNN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_RNN.fit(X_train_rnn, Y_train, epochs=20, validation_split=0.3)

In [70]:
def func1(result):
    if result >= 0.5 : return 1 
    return 0

In [74]:
Y_train_pred = pd.Series(model_RNN.predict(X_train_rnn).flatten()).apply(func1)
Y_test_pred  = pd.Series(model_RNN.predict(X_test_rnn ).flatten()).apply(func1)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


In [ ]:
print(classification_report(Y_train, Y_train_pred))

In [ ]:
print(classification_report(Y_test,  Y_test_pred))

# LSTM (Long Short-Term Memory) 

- RNN 모델의 장기의존문제를 해결하기 위해 고안된 RNN 계열의 모델
- ![img3](https://wikidocs.net/images/page/22888/vaniila_rnn_and_different_lstm_ver2.PNG)
- Seq의 길이가 긴 데이터에 대해 Cell State라는 Node를 활용하여, 장기적으로 정보가 전달 될 수 있도록 정보를 저장 하는 Node를 추가한 형태
- LSTM은 Cell State를 조정하기 위해 세가지 주요 Gate(입력값이 처리되는 Filter)를 사용
  - 1. Input Gate (입력 게이트) : Cell State에 새로운 정보를 저장할지 말지를 결정
       - 하나는 Sigmoid 함수로 구성 -> 어떤 값을 Update 할 지를 결정
       - 다른 하나는 Tanh 함수로 구성 -> Cell State에 추가될 후보 값들을 생성
       - ![imge1](https://wikidocs.net/images/page/22888/inputgate.PNG)
  - 2. Forget Gate (삭제 게이트) : Cell State에서 어떤 정보를 제거할 지 결정
       - 현재 Input(입력)과 이전 Node에 있는 State 중 Sigmoid를 이용하여, 0과 1사이 값을 출력
       - 1 : 정보를 유지 / 0 : 정보를 삭제
       - ![imge1](https://wikidocs.net/images/page/22888/forgetgate.PNG)
  - 3. Output Gate (출력 게이트) : Cell State의 정보를 기반으로 어떤 값을 출력할지를 결정
       - Sigmoid 함수를 이용해 어떤 Cell State를 출력할지 결정하고, Cell State는 Tanh를 통과하여 값의 범위를 조정한 뒤 Sigmoid Node와 합쳐져 최종 출력을 계산
       - ![imge2](https://wikidocs.net/images/page/22888/outputgateandhiddenstate.PNG)

In [77]:
from keras.layers import LSTM 

- Drop Out : 신경망을 학습할 때, 과적합 현상을 방지하기 위한 규제 방법 / 일정 비율의 Node에 대한 Weight Update를 비활성화 

In [ ]:
model_LSTM = Sequential()
model_LSTM.add( Embedding(10000, 8))
model_LSTM.add( LSTM(128, dropout=0.2 ))
model_LSTM.add( Dense(1, activation='sigmoid'))
model_LSTM.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_LSTM.fit(X_train_rnn, Y_train, epochs=20, validation_split=0.3)

In [79]:
Y_train_pred = pd.Series(model_LSTM.predict(X_train_rnn).flatten()).apply(func1)
Y_test_pred  = pd.Series(model_LSTM.predict(X_test_rnn ).flatten()).apply(func1)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


In [ ]:
print(classification_report(Y_train, Y_train_pred))

In [ ]:
print(classification_report(Y_test,  Y_test_pred))

# GRU (Gated Recurrent Unit)
- LSTM 의 변형 형태로, LSTM 모델의 3가지 게이트 구조를 단순화하여 효율적인 계산 (시간과 자원)
- ![img4](https://wikidocs.net/images/page/22889/gru.PNG)

- Update Gate : 현재 계산된 State값을 다음 Node에 얼마나 반영할 지를 계산
    - LSTM 모델에 Input Gate 와 Forget Gate을 수행
    - 과거의 정보를 얼마나 유지하고, 다음 정보를 전달할지를 결정 
- Reset Gate : 이전에 Node에서 전달받은 State가 현재의 입력값과 함께 어떻게 연산 될지를 결정
      - LSTM에 Cell State 없이, 과거의 정보를 효과적으로 필터링
- LSTM 알고리즘과 비교했을 때, 더 적은 Parameter(Layer, Node, Time)로 잘 작동 됨

In [82]:
from keras.layers import GRU 

In [ ]:
model_GRU = Sequential()
model_GRU.add( Embedding(10000,8))
model_GRU.add( GRU(128, dropout=0.2) )
model_GRU.add( Dense(1, activation='sigmoid'))
model_GRU.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_GRU.fit(X_train_rnn, Y_train, epochs=20, validation_split=0.3)

In [85]:
Y_train_pred = pd.Series(model_GRU.predict(X_train_rnn).flatten()).apply(func1)
Y_test_pred  = pd.Series(model_GRU.predict(X_test_rnn ).flatten()).apply(func1)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


In [ ]:
print(classification_report(Y_train, Y_train_pred))

In [87]:
print(classification_report(Y_test,  Y_test_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.84      1.00      0.91        16

    accuracy                           0.84        19
   macro avg       0.42      0.50      0.46        19
weighted avg       0.71      0.84      0.77        19



C:\Users\CONET-02\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\CONET-02\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\CONET-02\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
